In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import xarray as xr

from scipy.interpolate import griddata

# for reasonably clean early exit from cells - use exit()
import sys
sys.path.append("..")
from ipython_exit import exit

# Import the library EUMDAC
import eumdac
import time
import requests
import fnmatch
import shutil

from IPython.core.display import HTML
import datetime

2023-10-30 14:24:29.113851: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


# Authentication

In [2]:
# Insert your personal key and secret into the single quotes
# https://api.eumetsat.int/api-key/

consumer_key = 'JSXRnY1_XgcYeOox8PjKozflwrMa'
consumer_secret = 'fbKReD9xf6vh7iE9R5lVVtBID0Ua'

credentials = (consumer_key, consumer_secret)

token = eumdac.AccessToken(credentials)

try:
    print(f"This token '{token}' expires {token.expiration}")
except requests.exceptions.HTTPError as exc:
    print(f"Error when trying the request to the server: '{exc}'")

data_dir = "/lustre/storeB/project/IT/geout/machine-ocean/data_raw/metop/"

This token 'add91be5-3ef5-32db-8989-ffe8dbcb0dc2' expires 2023-10-30 15:24:55.755323


# Examples of low-resolution images collocated with Irminger 6

In [5]:
with open('../in_situ_obs_ascat_with_customisations.pickle', 'rb') as handle: 
    ascat_dict = pickle.load(handle)    

In [6]:
ascat_lon = ascat_dict['Irminger_6']['lon'][0]
ascat_lat = ascat_dict['Irminger_6']['lat'][0]
print(ascat_lon)
print(ascat_lat)

-39.47378
59.9337


In [7]:
products = ascat_dict['Irminger_6']['products']
products

<class 'eumdac.collection.SearchResults'>(EO:EUM:DAT:METOP:ASCSZR1B, {'publication': None, 'dtstart': '2019-08-11T00:10:59.996491+00:00', 'set': None, 'bbox': None, 'orbit': None, 'type': None, 'sat': None, 'sort': None, 'title': None, 't6': None, 'geo': 'POINT(-39.4738 59.9337)', 'dtend': '2020-01-22T09:11:00.009255+00:00', 'zone': None})

In [8]:
dict_low_res = {}
dict_expected_res = {}
count_low = 0
count_high = 0
for key, ascat_fn in ascat_dict['Irminger_6']['nc_files'].items():
    data_ascat = xr.open_dataset(data_dir + ascat_fn)
    if data_ascat.lat[0] - data_ascat.lat[1] > 2:
        dict_low_res[key] = ascat_fn
        print(key, ascat_fn)
        count_low = count_low + 1
    else:
        dict_expected_res[key] = ascat_fn
        #print(ascat_fn)
        count_high = count_high + 1
print(count_low)
print(count_high)

ASCA_SZR_1B_M03_20200121221800Z_20200122000258Z_N_O_20200122000039Z ASCATL1SZR_20200121T221800Z_20200122T000258Z_epct_dde0d061_P.nc
ASCA_SZR_1B_M02_20200121214800Z_20200121232958Z_N_O_20200121233009Z ASCATL1SZR_20200121T214800Z_20200121T232958Z_epct_f73691bd_P.nc
ASCA_SZR_1B_M03_20200120224200Z_20200121002358Z_N_O_20200121002312Z ASCATL1SZR_20200120T224200Z_20200121T002358Z_epct_ecdb2691_P.nc
ASCA_SZR_1B_M01_20200119220300Z_20200119234758Z_N_O_20200119225243Z ASCATL1SZR_20200119T220300Z_20200119T234758Z_epct_e8dea66a_P.nc
ASCA_SZR_1B_M01_20200118222400Z_20200119000858Z_N_O_20200118231342Z ASCATL1SZR_20200118T222400Z_20200119T000858Z_epct_dd7cfae2_P.nc
ASCA_SZR_1B_M03_20200117220300Z_20200117234458Z_N_O_20200117234419Z ASCATL1SZR_20200117T220300Z_20200117T234458Z_epct_42e14e13_P.nc
ASCA_SZR_1B_M02_20200117213000Z_20200117231158Z_N_O_20200117231220Z ASCATL1SZR_20200117T213000Z_20200117T231158Z_epct_5ed3fa55_P.nc
ASCA_SZR_1B_M03_20200116222400Z_20200117000558Z_N_O_20200117000541Z ASCATL1S

# Customisation

In [15]:
datatailor = eumdac.DataTailor(token)

In [16]:
chain = eumdac.tailor_models.Chain(
     product='ASCATL1SZR',
     format='netcdf4'
)

In [17]:
customisations = []

print("Running customisations")

i = 0

for product in products:
    while i < 5:
        print(i)
        # do not process products that are already downloaded
        if str(product) in dict_low_res.keys():     

            customisation = datatailor.new_customisation(product, chain)

            try:
                print(f"Customisation {customisation._id} started.")
            except eumdac.datatailor.DataTailorError as error:
                print(f"Error related to the Data Tailor: '{error.msg}'")
                break
            except requests.exceptions.HTTPError as error:
                print(f"HTTP error: '{error.msg}'")
                break
            except requests.exceptions.RequestException as error:
                print(f"Unexpected error: {error}")
                break

            customisations.append(customisation)

            status = customisation.status
            sleep_time = 10 # seconds

            # Customisation Loop
            while status:
                # Get the status of the ongoing customisation
                status = customisation.status

                if "DONE" in status:
                    print(f"Customisation {customisation._id} is successfully completed.")
                    break
                elif status in ["ERROR","FAILED","DELETED","KILLED","INACTIVE"]:
                    print(f"Customisation {customisation._id} was unsuccessful. Customisation log is printed.\n")
                    print(customisation.logfile)
                    break
                elif "QUEUED" in status:
                    print(f"Customisation {customisation._id} is queued.")
                elif "RUNNING" in status:
                    print(f"Customisation {customisation._id} is running.")
                time.sleep(sleep_time)
            i = i + 1
                
print("All customisations completed!")

Running customisations
0
Customisation bd126fe6 started.
Customisation bd126fe6 is running.
Customisation bd126fe6 is successfully completed.
1
Customisation 8f8ce6c7 started.
Customisation 8f8ce6c7 is running.
Customisation 8f8ce6c7 is running.
Customisation 8f8ce6c7 is successfully completed.
2
Customisation 363a3ff0 started.
Customisation 363a3ff0 is running.
Customisation 363a3ff0 is running.
Customisation 363a3ff0 is successfully completed.
3
Customisation 80c6b4ea started.
Customisation 80c6b4ea is running.
Customisation 80c6b4ea is running.
Customisation 80c6b4ea is successfully completed.
4
Customisation 79ef313d started.
Customisation 79ef313d is running.
Customisation 79ef313d is running.
Customisation 79ef313d is successfully completed.
All customisations completed!


In [18]:
customisations

[<class 'eumdac.customisation.Customisation'>(bd126fe6),
 <class 'eumdac.customisation.Customisation'>(8f8ce6c7),
 <class 'eumdac.customisation.Customisation'>(363a3ff0),
 <class 'eumdac.customisation.Customisation'>(80c6b4ea),
 <class 'eumdac.customisation.Customisation'>(79ef313d)]

# Download

In [21]:
print("Downloading products")

for customisation in customisations:
    jobID = customisation._id
    print(f"\tStarting to download the NetCDF output of the customisation {jobID}")

    nc, = fnmatch.filter(customisation.outputs, '*.nc')
    try:
        with customisation.stream_output(nc,) as stream, \
                open('ascat_examples_without_reprojecting/netcdf4/' + stream.name, mode='wb') as fdst:
            shutil.copyfileobj(stream, fdst)
        print(f"Dowloaded the output of the customisation {jobID}")
    except eumdac.datatailor.CustomisationError as error:
        print(f"Data Tailor Error", error)
        break
    except requests.exceptions.HTTPError as error:
        print(f"HTTP error: '{error.msg}'")
        break
    except requests.exceptions.RequestException as error:
        print(f"Unexpected error: {error}")
        break

    try:
        customisation.delete()
    except eumdac.datatailor.CustomisationError as exc:
        print("Customisation Error:", exc)
        break
    except requests.exceptions.HTTPError as error:
        print(f"HTTP error: '{error.msg}'")
        break
    except requests.exceptions.RequestException as error:
        print("Unexpected error:", error)
        break

    print(f"\tFinished dowloading the NetCDF output and deleted the customisation {jobID}")

# pickle dict with nc filenames
#with open('in_situ_obs_ascat_with_customisations.pickle', 'wb') as handle:
#        print("Saving dict!")
#        pickle.dump(in_situ_obs, handle, protocol=pickle.HIGHEST_PROTOCOL)

	Starting to download the NetCDF output of the customisation bd126fe6
Dowloaded the output of the customisation bd126fe6
	Finished dowloading the NetCDF output and deleted the customisation bd126fe6
	Starting to download the NetCDF output of the customisation 8f8ce6c7
Dowloaded the output of the customisation 8f8ce6c7
	Finished dowloading the NetCDF output and deleted the customisation 8f8ce6c7
	Starting to download the NetCDF output of the customisation 363a3ff0
Dowloaded the output of the customisation 363a3ff0
	Finished dowloading the NetCDF output and deleted the customisation 363a3ff0
	Starting to download the NetCDF output of the customisation 80c6b4ea
Dowloaded the output of the customisation 80c6b4ea
	Finished dowloading the NetCDF output and deleted the customisation 80c6b4ea
	Starting to download the NetCDF output of the customisation 79ef313d
Dowloaded the output of the customisation 79ef313d
	Finished dowloading the NetCDF output and deleted the customisation 79ef313d


# Customisation

In [27]:
datatailor = eumdac.DataTailor(token)

In [28]:
chain = eumdac.tailor_models.Chain(
     product='ASCATL1SZR',
     format='netcdf4_satellite'
)

In [29]:
customisations = []

print("Running customisations")

i = 0

for product in products:
    while i < 5:
        print(i)
        # do not process products that are already downloaded
        if str(product) in dict_low_res.keys():     

            customisation = datatailor.new_customisation(product, chain)

            try:
                print(f"Customisation {customisation._id} started.")
            except eumdac.datatailor.DataTailorError as error:
                print(f"Error related to the Data Tailor: '{error.msg}'")
                break
            except requests.exceptions.HTTPError as error:
                print(f"HTTP error: '{error.msg}'")
                break
            except requests.exceptions.RequestException as error:
                print(f"Unexpected error: {error}")
                break

            customisations.append(customisation)

            status = customisation.status
            sleep_time = 10 # seconds

            # Customisation Loop
            while status:
                # Get the status of the ongoing customisation
                status = customisation.status

                if "DONE" in status:
                    print(f"Customisation {customisation._id} is successfully completed.")
                    break
                elif status in ["ERROR","FAILED","DELETED","KILLED","INACTIVE"]:
                    print(f"Customisation {customisation._id} was unsuccessful. Customisation log is printed.\n")
                    print(customisation.logfile)
                    break
                elif "QUEUED" in status:
                    print(f"Customisation {customisation._id} is queued.")
                elif "RUNNING" in status:
                    print(f"Customisation {customisation._id} is running.")
                time.sleep(sleep_time)
            i = i + 1
                
print("All customisations completed!")

Running customisations
0
Customisation 5b75c28d started.
Customisation 5b75c28d is running.
Customisation 5b75c28d is running.
Customisation 5b75c28d is running.
Customisation 5b75c28d is successfully completed.
1
Customisation 73ff5e29 started.
Customisation 73ff5e29 is running.
Customisation 73ff5e29 is running.
Customisation 73ff5e29 is running.
Customisation 73ff5e29 is successfully completed.
2
Customisation 1e5fa23f started.
Customisation 1e5fa23f is running.
Customisation 1e5fa23f is running.
Customisation 1e5fa23f is successfully completed.
3
Customisation dfb5d145 started.
Customisation dfb5d145 is running.
Customisation dfb5d145 is running.
Customisation dfb5d145 is successfully completed.
4
Customisation 4c66b60b started.
Customisation 4c66b60b is running.
Customisation 4c66b60b is running.
Customisation 4c66b60b is successfully completed.
All customisations completed!


In [30]:
customisations

[<class 'eumdac.customisation.Customisation'>(5b75c28d),
 <class 'eumdac.customisation.Customisation'>(73ff5e29),
 <class 'eumdac.customisation.Customisation'>(1e5fa23f),
 <class 'eumdac.customisation.Customisation'>(dfb5d145),
 <class 'eumdac.customisation.Customisation'>(4c66b60b)]

# Download

In [31]:
print("Downloading products")

i = 0
for customisation in customisations:
    jobID = customisation._id
    print(f"\tStarting to download the NetCDF output of the customisation {jobID}")

    nc, = fnmatch.filter(customisation.outputs, '*.nc')
    try:
        with customisation.stream_output(nc,) as stream, \
                open('ascat_examples_without_reprojecting/netcdf4_satellite/' + stream.name + '_' + str(i), mode='wb') as fdst:
            shutil.copyfileobj(stream, fdst)
        print(f"Dowloaded the output of the customisation {jobID}")
    except eumdac.datatailor.CustomisationError as error:
        print(f"Data Tailor Error", error)
        break
    except requests.exceptions.HTTPError as error:
        print(f"HTTP error: '{error.msg}'")
        break
    except requests.exceptions.RequestException as error:
        print(f"Unexpected error: {error}")
        break

    try:
        customisation.delete()
    except eumdac.datatailor.CustomisationError as exc:
        print("Customisation Error:", exc)
        break
    except requests.exceptions.HTTPError as error:
        print(f"HTTP error: '{error.msg}'")
        break
    except requests.exceptions.RequestException as error:
        print("Unexpected error:", error)
        break

    print(f"\tFinished dowloading the NetCDF output and deleted the customisation {jobID}")
    i = i + 1

# pickle dict with nc filenames
#with open('in_situ_obs_ascat_with_customisations.pickle', 'wb') as handle:
#        print("Saving dict!")
#        pickle.dump(in_situ_obs, handle, protocol=pickle.HIGHEST_PROTOCOL)

	Starting to download the NetCDF output of the customisation 5b75c28d
Dowloaded the output of the customisation 5b75c28d
	Finished dowloading the NetCDF output and deleted the customisation 5b75c28d
	Starting to download the NetCDF output of the customisation 73ff5e29
Dowloaded the output of the customisation 73ff5e29
	Finished dowloading the NetCDF output and deleted the customisation 73ff5e29
	Starting to download the NetCDF output of the customisation 1e5fa23f
Dowloaded the output of the customisation 1e5fa23f
	Finished dowloading the NetCDF output and deleted the customisation 1e5fa23f
	Starting to download the NetCDF output of the customisation dfb5d145
Dowloaded the output of the customisation dfb5d145
	Finished dowloading the NetCDF output and deleted the customisation dfb5d145
	Starting to download the NetCDF output of the customisation 4c66b60b
Dowloaded the output of the customisation 4c66b60b
	Finished dowloading the NetCDF output and deleted the customisation 4c66b60b
